In [2]:
using Plots, Distributions, Random
using Parameters, LinearAlgebra, DataFrames, CSV
using Optim, BenchmarkTools
Random.seed!(1234)


default_p = @with_kw (
    # For Blue-Collar (option 1)
        e11=0.0938, # educational return
        e12=0.1170, # own experience return
        e1c2=0.0748, # complementarity return from white-collar experience
        e1c3=0.0077, # complementarity return from military experience
        e13=0.0461, # own experience quadratic term (negative term in reward function)
    # For White-Collar (option 2)
        e21=0.0189, # educational return
        e22=0.1424, # own experience return
        e2c1=0.0674, # complementarity return from blue-collar experience
        e2c3=0.1021, # complementarity return from military experience
        e23=0.1774, # own experience quadratic term (negative term in reward function)
    # For military (option 3)
        e31=0.0443, # educational return
        e32=0.3391, # own experience return
        e33=2.990,  # own experience quadratic term (negative term in reward function)
    # For students (option 4)
        tc1=2983, # secondary attending cost  (from appendix B)
        tc2=26357, # graduate attending cost (from appendix B)
        δ=0.7870, # school progression prob and discount factor
    # Rental prices
        r1=1.0, # market rental price of blue-collar
        r2=1.0, # market rental price of white-collar
        r3=1.0,  # market rental price of military
    # Starting and maximum age 
        start_age=16, # Starting age
        max_age=30, # Maximum age
    # Ex-ante unobserved heterogeneity (mass points)
        μ1=8.8043,
        μ2=9.85,
        μ3=9.5,
        μ4=43948,
        μ5=6887,
        e16=9, 
    # Shock standard deviations and correlations
        coef_m1_err_sd=0.3301,
        coef_m2_err_sd=0.3329,
        coef_m3_err_sd=0.3308,
        coef_m4_err_sd=2312,
        coef_m5_err_sd=13394,

        coef_m1_cor_m2=0.1226,
        coef_m1_cor_m3=0.0182,
        coef_m2_cor_m3=0.4727, 
    # Simulation block
        idsim=30, # Simulated individuals
        nsim=100 # Simulated shocks (for integration purposes)

)         

## Reward function

"""
    reward(e16,g,x1,x2,x3,m;p)
Function to compute the reward. 

# Arguments

- `e16::Float64`: stock of education at 16 yo.
- `g::Float64`: years of education at t.
- `x1::Float64`: experience in blue-collar occupation.
- `x2::Float64`: experience in white-collar occupation.
- `x3::Float64`: experience in military occupation.

"""
function reward(g,x1,x2,x3,m,ϵ;p)

    @unpack e11, e12, e1c2, e1c3, e13, e21, e22, e2c1, e2c3, e23, e31, e32, e33, tc1, tc2, r1, r2, r3, μ1,μ2,μ3,μ4,μ5 = p;

    if m==1
        r1*exp.((μ1+e11*g+e12*x1+e1c2*x2+e1c3*x3-e13*((x1^2)/1000)).+ϵ)
    elseif m==2
        r2*exp.((μ2+e21*g+e22*x2+e2c1*x1+e2c3*x3-e23*((x2^2)/1000)).+ϵ)
    elseif m==3
        r3*exp.((μ3+e31*g+e32*x3-e33*((x3^2)/1000)).+ϵ)
    elseif m==4
        (μ4-tc1*Int(g>=12)-tc2*Int(g>=16)).+ϵ
    else
        μ5.+ϵ
    end
end

p=default_p(); # create an instance for default parameters
errors=[1,1,1,1,1]
reward(13,3,2,1,1,errors;p)

function total_utility(u,ev;p)
    @unpack δ=p
    u.+δ*ev
end

@unpack max_age,start_age=default_p()
n=max_age-start_age+1

@unpack coef_m1_err_sd,coef_m2_err_sd,coef_m3_err_sd,coef_m4_err_sd,coef_m5_err_sd,coef_m1_cor_m2, coef_m1_cor_m3, coef_m2_cor_m3, idsim, nsim=default_p()

ar_sd = [coef_m1_err_sd, coef_m2_err_sd, coef_m3_err_sd, coef_m4_err_sd, coef_m5_err_sd] # Vector of standard deviations
mt_cor =    [1 coef_m1_cor_m2 coef_m1_cor_m3 0 0; #= correlations=#
            coef_m1_cor_m2 1 coef_m2_cor_m3 0 0;
            coef_m1_cor_m3 coef_m2_cor_m3 1 0 0;
            0 0 0 1 0;
            0 0 0 0 1]


mat_varcov = zeros(5,5)
mat_varcov[diagind(mat_varcov)]=ar_sd.^2
mat_varcov[1,2] = mt_cor[1,2]*ar_sd[1]*ar_sd[2];
mat_varcov[2,1] = mat_varcov[1,2];
mat_varcov[1,3] = mt_cor[1,3]*ar_sd[1]*ar_sd[3];
mat_varcov[3,1] = mat_varcov[1,3];
mat_varcov[2,3] = mt_cor[2,3]*ar_sd[2]*ar_sd[3];
mat_varcov[3,2] = mat_varcov[2,3];

μ=[0,0, 0, 0, 0] # Mean of shocks
Σ=(Hermitian(mat_varcov)) # Hermitian matrix of variance and covariance
d=MvNormal(μ,Σ) # Distribution of shocks

### Total number of draws
N=nsim
### Total number of simulated histories
M=idsim*max_age

eps_mat=rand(d,N*5)' # Draw of shocks
#= This was wrong!

eps_m1=eps_mat[(N*0+1):N*1,1]
eps_m2=eps_mat[(N*1+1):N*2,2]
eps_m3=eps_mat[(N*2+1):N*3,3]
eps_m4=eps_mat[(N*3+1):N*4,4]
eps_m5=eps_mat[(N*4+1):N*5,5]
=#
eps_m1=eps_mat[(N*0+1):N*1,1]
eps_m2=eps_mat[(N*1+1):N*2,2]
eps_m3=eps_mat[(N*2+1):N*3,3]
eps_m4=eps_mat[(N*3+1):N*4,4]
eps_m5=eps_mat[(N*4+1):N*5,5]

@unpack e16=default_p() # Unpacking education at age 16

#= Solving the Expected Value Function
We have 4 state variables, i.e., occupations (3) + education (1), so 
we need to compute a 4-dimensional matrix for each age. Therefore,
there will be max_age-4-dimensional-matrixes. I store this matrix
into an array object. 
=#
floor_min_prob=0.0001; # To avoid null mass for some options.
@time begin

# Outer loop
EV=Any[]; # Array to save expected value. Similar to cell in Matlab
prob_d_m1=Any[]; # Array to save prob of choosing m1. Similar to cell in Matlab
prob_d_m2=Any[]; # Array to save prob of choosing m2. Similar to cell in Matlab
prob_d_m3=Any[]; # Array to save prob of choosing m3. Similar to cell in Matlab
prob_d_m4=Any[]; # Array to save prob of choosing m4. Similar to cell in Matlab
prob_d_m5=Any[]; # Array to save prob of choosing m5. Similar to cell in Matlab

for t=max_age:-1:1 # We solve backward.

    mn_EV_t_ED = fill(NaN,t,t,t,t);
    mn_prob_d_m1 = fill(NaN,t,t,t,t);
    mn_prob_d_m2 = fill(NaN,t,t,t,t);
    mn_prob_d_m3 = fill(NaN,t,t,t,t);
    mn_prob_d_m4 = fill(NaN,t,t,t,t);
    mn_prob_d_m5 = fill(NaN,t,t,t,t);

    i=1 # This is not necessary, I keep it because it help me to find errors in my code.

    # Inner loop
    for g=0:t-1, x1=0:t-1, x2=0:t-1, x3=0:t-1 # Improves readability. 
        if g + x1 + x2 + x3 <= t # Conditional nested-loop. Here, we constraint the loop into feasible options.

            #= current utility. 
            This is a vector because we need to solve the integral.
            If we increase the number of simulated vectors, the integration
            will be more accurate.
            =#
            R_m1=reward(g+e16,x1,x2,x3,1,eps_m1;p);
            R_m2=reward(g+e16,x1,x2,x3,2,eps_m2;p);
            R_m3=reward(g+e16,x1,x2,x3,3,eps_m3;p);
            R_m4=reward(g+e16,x1,x2,x3,4,eps_m4;p);
            R_m5=reward(g+e16,x1,x2,x3,5,eps_m5;p);
            
            # For final age, there is no expected value.
            if (t==max_age)
                EV_m1=0;
                EV_m2=0;
                EV_m3=0;
                EV_m4=0;
                EV_m5=0;
            else
                mt_EV=EV[max_age-t]
                #= This part is a trick to solve the model only for maximum age,
                    without having compilations errors.
                    =#
                #EV_m1=0;
                #EV_m2=0;
                #EV_m3=0;
                #EV_m4=0;
                #EV_m5=0;
                EV_m1 = mt_EV[(g+1)+0, (x1+1)+1, (x2+1)+0, (x3+1)+0];
                EV_m2 = mt_EV[(g+1)+0, (x1+1)+0, (x2+1)+1, (x3+1)+0];
                EV_m3 = mt_EV[(g+1)+0, (x1+1)+0, (x2+1)+0, (x3+1)+1];
                EV_m4 = mt_EV[(g+1)+1, (x1+1)+0, (x2+1)+0, (x3+1)+0];
                EV_m5 = mt_EV[(g+1)+0, (x1+1)+0, (x2+1)+0, (x3+1)+0];

            end

            # Total utility
            U_m1 = total_utility(R_m1, EV_m1;p);
            U_m2 = total_utility(R_m2, EV_m2;p);
            U_m3 = total_utility(R_m3, EV_m3;p);
            U_m4 = total_utility(R_m4, EV_m4;p);
            U_m5 = total_utility(R_m5, EV_m5;p);

            # Finding choices
            U_m5_highest = (U_m5.>= U_m4).*(U_m5.>= U_m3).*(U_m5.>= U_m2).*(U_m5.>= U_m1);
            U_m4_highest = (U_m4.> U_m5).*(U_m4.> U_m3).*(U_m4.> U_m2).*(U_m4.> U_m1);
            U_m3_highest = (U_m3.> U_m5).*(U_m3.> U_m4).*(U_m3.> U_m2).*(U_m3.> U_m1);
            U_m2_highest = (U_m2.> U_m5).*(U_m2.> U_m4).*(U_m2.> U_m3).*(U_m2.> U_m1);
            U_m1_highest = (U_m1.> U_m5).*(U_m1.> U_m4).*(U_m1.> U_m3).*(U_m1.> U_m2);
            
            
            # Finding frequencies
            it_m5_opti = sum(U_m5_highest);
            it_m4_opti = sum(U_m4_highest);
            it_m3_opti = sum(U_m3_highest);
            it_m2_opti = sum(U_m2_highest);
            it_m1_opti = N-it_m5_opti-it_m4_opti-it_m3_opti-it_m2_opti;
            #= This part is another checkpoint. No need to have it, but
            I found it very convenient because I can keep track of each
            iteration and check the feasibility constraint.
            =#
            #println("Iteration #$i, chose 5=$it_m5_opti, g=$g,x1=$x1,x2=$x2,x3=$x3")
            #println("Iteration #$i, chose 4=$it_m4_opti, g=$g,x1=$x1,x2=$x2,x3=$x3")
            #println("Iteration #$i, chose 3=$it_m3_opti, g=$g,x1=$x1,x2=$x2,x3=$x3")
            #println("Iteration #$i, chose 2=$it_m2_opti, g=$g,x1=$x1,x2=$x2,x3=$x3")
            #println("Iteration #$i, chose 1=$it_m1_opti, g=$g,x1=$x1,x2=$x2,x3=$x3")

            # Finding probabilities
            prob_m5 = it_m5_opti/N+floor_min_prob;
            prob_m4 = it_m4_opti/N+floor_min_prob;
            prob_m3 = it_m3_opti/N+floor_min_prob;
            prob_m2 = it_m2_opti/N+floor_min_prob;
            prob_m1= (1+floor_min_prob*5)-prob_m5-prob_m4-prob_m3-prob_m2;
            #println("Iteration #$i, prob5=$prob_m5, g=$g,x1=$x1,x2=$x2,x3=$x3")
            #println("Iteration #$i, prob4=$prob_m4, g=$g,x1=$x1,x2=$x2,x3=$x3")

            # Expected value
            sum_m5_condi_opti = (U_m5_highest'*R_m5);
            sum_m4_condi_opti = (U_m4_highest'*R_m4);
            sum_m3_condi_opti = (U_m3_highest'*R_m3);
            sum_m2_condi_opti = (U_m2_highest'*R_m2);
            sum_m1_condi_opti = (U_m1_highest'*R_m1);
            EmaxV_simu = (sum_m5_condi_opti + sum_m4_condi_opti + sum_m3_condi_opti + sum_m2_condi_opti + sum_m1_condi_opti)/N;

            # Preparing to save. Remember to add 1 because of index.
            mn_prob_d_m1[g+1, x1+1, x2+1, x3+1] = prob_m1;
            mn_prob_d_m2[g+1, x1+1, x2+1, x3+1] = prob_m2;
            mn_prob_d_m3[g+1, x1+1, x2+1, x3+1] = prob_m3;
            mn_prob_d_m4[g+1, x1+1, x2+1, x3+1] = prob_m4;
            mn_prob_d_m5[g+1, x1+1, x2+1, x3+1] = prob_m5;
            mn_EV_t_ED[g+1, x1+1, x2+1, x3+1] = EmaxV_simu;
            i=i+1
        end
    end
    #= FINAL ARRAYS. Comments

    # Very Important: Index max_age refers to age 1, index max_age-1 refers to age 2.
    This means that if we work with a maximum age of 30 (46 years), the matrix
    EV[30] is actually the matrix at age 1 (16 years). The matrix for age 30 (46) will be
    EV[1].Is it possible to change this with a dictionary, if so, remember to change
    lines calling for this representation.

    @eval $(Symbol(:EV,t))=mn_EV_t_ED # This is another way, but is highly unrecommended
    because i) is unreadable, ii) is inefficient, iii) uses too much memory. For a broader
    discussiion, check:
    https://discourse.julialang.org/t/how-to-change-the-name-of-a-variable-in-a-for-loop/28510
    I am using `push!'

    =#
    push!(EV,mn_EV_t_ED); 

    push!(prob_d_m1,mn_prob_d_m1./(1+floor_min_prob*5));
    push!(prob_d_m2,mn_prob_d_m2./(1+floor_min_prob*5));
    push!(prob_d_m3,mn_prob_d_m3./(1+floor_min_prob*5));
    push!(prob_d_m4,mn_prob_d_m4./(1+floor_min_prob*5));
    push!(prob_d_m5,mn_prob_d_m5./(1+floor_min_prob*5));

end
#= Simulation
=#
fill(NaN,max_age,idsim);
ID_mat = fill(NaN,max_age,idsim);
t_mat = fill(NaN,max_age,idsim);
# Matrixes of state variables
G_mat = fill(NaN,max_age,idsim);
X1_mat = fill(NaN,max_age,idsim);
X2_mat = fill(NaN,max_age,idsim);
X3_mat = fill(NaN,max_age,idsim);
# Matrix of choices
D_mat = fill(NaN,max_age,idsim);
# Returns of choices
R_m1_mat = fill(NaN,max_age,idsim);
R_m2_mat = fill(NaN,max_age,idsim);
R_m3_mat = fill(NaN,max_age,idsim);
R_m4_mat = fill(NaN,max_age,idsim);
R_m5_mat = fill(NaN,max_age,idsim);

# Idiosyncratic shock, individual-age specific.
eps_mat_sim=rand(d,M)' 

eps_m1=eps_mat_sim[:,1]
eps_m2=eps_mat_sim[:,2]
eps_m3=eps_mat_sim[:,3]
eps_m4=eps_mat_sim[:,4]
eps_m5=eps_mat_sim[:,5]

j=1; # This is the person-age counter.
for id_i=1:1:idsim
    for t=1:1:max_age

        if t==1
            G_mat[1, id_i] = 0;
            X1_mat[1, id_i] = 0;
            X2_mat[1, id_i] = 0;
            X3_mat[1, id_i] = 0;
        end
        g = G_mat[t, id_i];
        x1 = X1_mat[t, id_i];
        x2 = X2_mat[t, id_i];
        x3 = X3_mat[t, id_i];
        #= I have to re-convert to Integer, 
        because the matrix elements are Float64 and the index
        counter is defined over int.
        =#
        g=Int(g);
        x1=Int(x1);
        x2=Int(x2);
        x3=Int(x3);

        R_m1=reward(g+e16,x1,x2,x3,1,eps_m1[j];p);
        R_m2=reward(g+e16,x1,x2,x3,2,eps_m1[j];p);
        R_m3=reward(g+e16,x1,x2,x3,3,eps_m1[j];p);
        R_m4=reward(g+e16,x1,x2,x3,4,eps_m1[j];p);
        R_m5=reward(g+e16,x1,x2,x3,5,eps_m1[j];p);
        global j=j+1

        # Future utility
        if t==max_age
            EV_m1 = 0;
            EV_m2 = 0;
            EV_m3 = 0;
            EV_m4 = 0;
            EV_m5 = 0;
        else
            mt_EV = EV[max_age-t]; # Because the array is backwardly defined.
            #println(mt_EV)
            #println(g)
            EV_m1 = mt_EV[(g+1)+0, (x1+1)+1, (x2+1)+0, (x3+1)+0];
            EV_m2 = mt_EV[(g+1)+0, (x1+1)+0, (x2+1)+1, (x3+1)+0];
            EV_m3 = mt_EV[(g+1)+0, (x1+1)+0, (x2+1)+0, (x3+1)+1];
            EV_m4 = mt_EV[(g+1)+1, (x1+1)+0, (x2+1)+0, (x3+1)+0];
            EV_m5 = mt_EV[(g+1)+0, (x1+1)+0, (x2+1)+0, (x3+1)+0];            
        end

        # utility
        U_m1 = total_utility(R_m1, EV_m1;p);
        U_m2 = total_utility(R_m2, EV_m2;p);
        U_m3 = total_utility(R_m3, EV_m3;p);
        U_m4 = total_utility(R_m4, EV_m4;p);
        U_m5 = total_utility(R_m5, EV_m5;p);

        # Decisions
        if ((U_m5 > U_m4) && (U_m5 > U_m3) && (U_m5 > U_m2) && (U_m5 > U_m1))
            D_opti = 5;
        elseif ((U_m4 > U_m3) && (U_m4 > U_m2) && (U_m4 > U_m1))
            D_opti = 4;
        elseif ((U_m3 > U_m2) && (U_m3 > U_m1))
            D_opti = 3;
        elseif (U_m2 > U_m1)
            D_opti = 2;
        else
            D_opti = 1;
        end

        # Save into matrixes
        ID_mat[t,id_i]=id_i;
        t_mat[t, id_i] = t;

        if t<max_age

            # No new decisions, which means home-production.
            X1_mat[t+1, id_i] = x1;
            X2_mat[t+1, id_i] = x2;
            X3_mat[t+1, id_i] = x3;
            G_mat[t+1, id_i] = g;

            # If made a new decision, no home-production.
            if D_opti == 1
                X1_mat[t+1, id_i] = x1 + 1;
            elseif D_opti == 2
                X2_mat[t+1, id_i] = x2 + 1;
            elseif D_opti == 3
                X3_mat[t+1, id_i] = x3 + 1;
            elseif D_opti == 4
                G_mat[t+1, id_i] = g + 1;
            end
        end
            

        D_mat[t, id_i] = D_opti;
        R_m1_mat[t, id_i] = R_m1;
        R_m2_mat[t, id_i] = R_m2;
        R_m3_mat[t, id_i] = R_m3;
        R_m4_mat[t, id_i] = R_m4;
        R_m5_mat[t, id_i] = R_m5;

    end
end


end

## Preparing to export to cvs. Flatten matrixes

G_mat_aux=G_mat.+e16; # Auxiliar G matrix plus initial education at age 16
data_mat=   [ID_mat[:] t_mat[:] G_mat_aux[:] X1_mat[:] X2_mat[:] X3_mat[:] D_mat[:] R_m1_mat[:] R_m2_mat[:] R_m3_mat[:] R_m4_mat[:] R_m5_mat[:]];


 22.333803 seconds (61.04 M allocations: 7.534 GiB, 8.63% gc time, 0.80% compilation time)
